# Using the geocoder api to get GPS coordinates

Now that we have build a dataframe listing postcodes again borough and neighborhood we need longitude and lattitude data to be used to query the `foursquares` rest api.

We will be using the geocoder package for python3
+ Doc: https://geocoder.readthedocs.io/index.html
+ Install : ```pip3 install geocoder```

In [1]:
import pandas as pd
import geocoder

In [2]:
# Read our previous work on post codes.
pCodes_df = pd.read_csv('postCodes.csv')
pCodes_df

,Unnamed: 0,PostalCode,Borough,Neighborhood
0,0,M3A,North York,Parkwoods
1,1,M4A,North York,Victoria Village
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...
98,98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,99,M4Y,Downtown Toronto,Church and Wellesley
100,100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [4]:
def getCoordinates(postal_code, nTry=10):
    '''    
    Return the GPS coordinates (long, lat) of a postal code.
            Parameters:
                    postal_code (str): A postal code
                    nTry (int): The number of time the geocoder API should be queried

            Returns:
                    gps_coordinates (float, float): Binary string of the sum of a and b
    '''  
    
    lat_lng_coords = None    

    # loop until you get the coordinates
    while lat_lng_coords is None and nTry != 0:
        
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        
        nTry -= 1
        
        
    if not lat_lng_coords is None:
        return (lat_lng_coords[0], lat_lng_coords[1])
    else:
        # If failed to get Coordinates after nTry iterations
        # Read the csv file, return coordinates
        coord_df = pd.read_csv('Geospatial_Coordinates.csv')
        return tuple(map(float, [coord_df[coord_df['Postal Code'] == postal_code].Latitude, coord_df[coord_df['Postal Code'] == postal_code].Longitude]))

In [5]:
# Testing the function wit a postcode that should be mapped to coordinates 43.66, -79.38
print(getCoordinates('M4Y', nTry=0))

(43.6658599, -79.38315990000001)


In [6]:
pCodesCoord_df = pCodes_df
pCodesCoord_df

,Unnamed: 0,PostalCode,Borough,Neighborhood
0,0,M3A,North York,Parkwoods
1,1,M4A,North York,Victoria Village
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...
98,98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,99,M4Y,Downtown Toronto,Church and Wellesley
100,100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [7]:
# Augmenting our dataframe with freshly fetched gps coordinates.
coords = [getCoordinates(pc, nTry=0) for pc in list(pCodesCoord_df.PostalCode)] 

longs = [c[0] for c in coords]
lats  = [c[1] for c in coords]

pCodesCoord_df['Latitude'], pCodesCoord_df['Longitude']  = longs, lats
pCodesCoord_df

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...,...
98,98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Exporting the dataFrame

In [8]:
pCodesCoord_df.to_csv('postCodes_Coords.csv')